# Toy modelling demonstration

This notebook demonstrates the use of several other scripts contained in this directory:
 - **[toy_data_generation.py](./toy_data_generation.py)**: generates "toy" datasets using known, specified generative processes
 - **[param_dict_generation.py](./param_dict_generation.py)**: generates parameter dicts specifying candidate models to train on the toy (or other, real) datasets
 - **[model_wrappers.py](./model_wrappers.py)**: presents a common interface for scikit-learn, statsmodels and XGBoost models, allowing multiple model types to be specified using the parameter dicts generated by param_dict_generation.py
 
The output is a csv file evaluating the performance of several different models on several toy datasets.

# Imports and settings

In [2]:
import multiprocessing
import pandas as pd
from sklearn.model_selection import train_test_split

from model_wrappers import try_models
from toy_data_generation import generate_linear_data
from param_dict_generation import generate_ols_list, generate_mlp_list, generate_xgb_list

In [3]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', None)

# Dataset generation

In [4]:
datasets = []
for nrows in [1000, 3000]:
    for nvars in [10, 30]:
        datasets.append((*generate_linear_data(nrows, nvars), f'linear_{nvars}vars_{nrows}rows'))
        print(f'Datasets with {nrows} rows and {nvars} vars successfully appended')

Datasets with 1000 rows and 10 vars successfully appended
Datasets with 1000 rows and 30 vars successfully appended
Datasets with 3000 rows and 10 vars successfully appended
Datasets with 3000 rows and 30 vars successfully appended


# Experiment list generation

## Statsmodels list

In [5]:
ols_list = generate_ols_list(alpha_list=[0.01, 1], L1_wt_list=[0, 0.5, 1])

statsmodels_param_list = ols_list

## Scikit list

In [6]:
n_cpus = multiprocessing.cpu_count()

In [7]:
mlp_list = generate_mlp_list(hidden_layout_list=[(200,), (50, 50)])

xgb_list = generate_xgb_list(depth_list = [3,5,7],
                             learning_rate_list = [0.03, 0.1, 0.3],
                             n_jobs=max(1, n_cpus-1),
                             objective='Linear',
                             early_stopping_rounds=10)

scikit_param_list = mlp_list

xgb_param_list = xgb_list

# Model training and evaluation

In [8]:
results = {}

for X, y, dataset_name in datasets:
    print(f'\n\n{dataset_name}\n')
    # update scikit_param_list for xgboost early stopping eval_sets if present
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=303)
    
    scikit_results = try_models(
            X_train, X_test, y_train, y_test,
            param_list=scikit_param_list, model='scikit'
        )
    for key in scikit_results:
        results[(dataset_name, key)] = scikit_results[key]
        
    xgb_results = try_models(
            X_train, X_test, y_train, y_test,
            param_list=xgb_param_list, model='xgb'
        )
    for key in xgb_results:
        results[(dataset_name, key)] = xgb_results[key]
        
    sm_results = try_models(
            X_train, X_test, y_train, y_test,
            param_list=statsmodels_param_list, model='statsmodels'
        )
    for key in sm_results:
        results[(dataset_name, key)] = sm_results[key]
    
display(pd.DataFrame(results).T)



linear_10vars_1000rows

MLP_200 trained in 0.65 seconds
MLP_50_50 trained in 0.83 seconds
XGB_Linear_maxd_3_lr_0.03 trained in 0.19 seconds
XGB_Linear_maxd_3_lr_0.1 trained in 0.03 seconds
XGB_Linear_maxd_3_lr_0.3 trained in 0.01 seconds
XGB_Linear_maxd_5_lr_0.03 trained in 0.07 seconds
XGB_Linear_maxd_5_lr_0.1 trained in 0.03 seconds
XGB_Linear_maxd_5_lr_0.3 trained in 0.01 seconds
XGB_Linear_maxd_7_lr_0.03 trained in 0.08 seconds
XGB_Linear_maxd_7_lr_0.1 trained in 0.03 seconds
XGB_Linear_maxd_7_lr_0.3 trained in 0.01 seconds
OLS_no_reg trained in 0.24 seconds
OLS_alpha_0.01_L1_0 trained in 0.04 seconds
OLS_alpha_0.01_L1_0.5 trained in 0.08 seconds
OLS_alpha_0.01_L1_1 trained in 0.06 seconds
OLS_alpha_1_L1_0 trained in 0.01 seconds
OLS_alpha_1_L1_0.5 trained in 0.02 seconds
OLS_alpha_1_L1_1 trained in 0.01 seconds


linear_30vars_1000rows

MLP_200 trained in 3.78 seconds
MLP_50_50 trained in 1.44 seconds
XGB_Linear_maxd_3_lr_0.03 trained in 0.29 seconds
XGB_Linear_maxd_3_lr_0.1 tra

mse_test  prediction_time  \
linear_10vars_1000rows MLP_200                    1.043281         0.001665   
                       MLP_50_50                  1.062724         0.001359   
                       XGB_Linear_maxd_3_lr_0.03  1.070450         0.002043   
                       XGB_Linear_maxd_3_lr_0.1   1.059702         0.001249   
                       XGB_Linear_maxd_3_lr_0.3   1.064197         0.001048   
                       XGB_Linear_maxd_5_lr_0.03  1.062643         0.001737   
                       XGB_Linear_maxd_5_lr_0.1   1.067359         0.001206   
                       XGB_Linear_maxd_5_lr_0.3   1.061659         0.000954   
                       XGB_Linear_maxd_7_lr_0.03  1.098140         0.001999   
                       XGB_Linear_maxd_7_lr_0.1   1.101398         0.001122   
                       XGB_Linear_maxd_7_lr_0.3   1.092107         0.000980   
                       OLS_no_reg                 1.036529         0.001408   
                       OLS_alpha_0.01_L1_0        1.069125         0.001306   
                       OLS_alpha_0.01_L1_0.5      1.068995         0.001388   
                       OLS_alpha_0.01_L1_1        1.074263         0.001237   
                       OLS_alpha_1_L1_0           2.485076         0.001097   
                       OLS_alpha_1_L1_0.5         3.088801         0.001431   
                       OLS_alpha_1_L1_1           3.831503         0.000986   
linear_30vars_1000rows MLP_200                    1.522749         0.001722   
                       MLP_50_50                  1.392776         0.001546   
                       XGB_Linear_maxd_3_lr_0.03  1.214607         0.003446   
                       XGB_Linear_maxd_3_lr_0.1   1.241170         0.001899   
                       XGB_Linear_maxd_3_lr_0.3   1.286779         0.001259   
                       XGB_Linear_maxd_5_lr_0.03  1.434742         0.003448   
                       XGB_Linear_maxd_5_lr_0.1   1.363875         0.001867   
                       XGB_Linear_maxd_5_lr_0.3   1.528355         0.001325   
                       XGB_Linear_maxd_7_lr_0.03  1.720478         0.004488   
                       XGB_Linear_maxd_7_lr_0.1   1.724918         0.001773   
                       XGB_Linear_maxd_7_lr_0.3   1.729163         0.001248   
                       OLS_no_reg                 0.985869         0.005932   
                       OLS_alpha_0.01_L1_0        0.993123         0.000916   
                       OLS_alpha_0.01_L1_0.5      1.077157         0.001285   
                       OLS_alpha_0.01_L1_1        1.077987         0.001259   
                       OLS_alpha_1_L1_0           5.282186         0.000942   
                       OLS_alpha_1_L1_0.5         6.760821         0.001102   
                       OLS_alpha_1_L1_1           7.179704         0.001072   
linear_10vars_3000rows MLP_200                    0.977037         0.003874   
                       MLP_50_50                  0.991620         0.002164   
                       XGB_Linear_maxd_3_lr_0.03  0.993793         0.004795   
                       XGB_Linear_maxd_3_lr_0.1   0.995134         0.002168   
                       XGB_Linear_maxd_3_lr_0.3   0.997584         0.001532   
                       XGB_Linear_maxd_5_lr_0.03  1.038890         0.003857   
                       XGB_Linear_maxd_5_lr_0.1   1.032579         0.001807   
                       XGB_Linear_maxd_5_lr_0.3   1.036664         0.001239   
                       XGB_Linear_maxd_7_lr_0.03  1.114709         0.004012   
                       XGB_Linear_maxd_7_lr_0.1   1.106716         0.001956   
                       XGB_Linear_maxd_7_lr_0.3   1.103412         0.001238   
                       OLS_no_reg                 0.965498         0.001265   
                       OLS_alpha_0.01_L1_0        0.974406         0.001072   
                       OLS_alpha_0.01_L1_0.5      0.972742         0.001245   
                     

# Output results

In [8]:
pd.DataFrame(results).T.to_csv('results.csv')